In [1]:
import pandas as pd
import numpy as np
import os

# Scientific question
how does transcription profile change with dengue disease?

goal: identify biomarkers

In [ ]:
data_df         = pd.read_csv(os.path.join("data", "dengue", "small_transcriptome", "denv.csv"), header=None).T
data_df.columns = data_df.iloc[0]
data_df         = data_df.iloc[1:]

# data_df = data_df.drop(columns=["Sample_organism_ch1", "Sample_type",
#                        "Sample_last_update_date", "Sample_contact_address",
#                        "Sample_contact_city",     "Sample_contact_state",
#                        "Sample_contact_country",  "Sample_supplementary_file", "series_matrix_table_end",
#                        'Sample_data_processing', 'Sample_platform_id', 'Sample_contact_name',
#                        'Sample_contact_email', 'Sample_contact_department'])

data_df["diagnosis"]  = data_df["diagnosis"].apply(lambda x: x.split(": ")[1])
data_df["patient_ID"] = data_df["Sample_title"].apply(lambda x: "-".join(x.split("-")[:-1]))

data_disease_df = data_df[["patient_ID", "diagnosis"]]
#data_disease_df = data_disease_df[data_disease_df.diagnosis != "exclude"]
data_disease_df["patient_diag"] = data_disease_df["patient_ID"] + "_" + data_disease_df["diagnosis"]
patient2diag_dict = dict(zip(data_disease_df["patient_ID"], data_disease_df["diagnosis"]))


In [ ]:
def modify_patient_id_gene_expression(patient_id):
    l = patient_id.split("-")
    if len(l) == 3:
        return "-".join(l[:-1])
    else:
        return "-".join(l)

gene_expression_df = pd.read_excel(os.path.join("data", "dengue",  "small_transcriptome", "denv_pcr.xlsx"))
gene_expression_df = gene_expression_df[gene_expression_df["patient_ID"]!="3-004-01"]

gene_expression_df["patient_ID"] = gene_expression_df["patient_ID"].apply(modify_patient_id_gene_expression)
gene_expression_df["diagnosis"]  = gene_expression_df["patient_ID"].apply(lambda x: patient2diag_dict[x])
gene_expression_df               = gene_expression_df[gene_expression_df["diagnosis"] != "exclude"]
gene_expression_df

In [ ]:
gene_expression_df[gene_expression_df.diagnosis=="healthy"]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


fig, ax = plt.subplots(2, 1, figsize=(10.2, 9.2))

ax[0].hist(np.log(gene_expression_df["Value"].values), edgecolor="w", facecolor="salmon",
                                                                alpha=0.5, bins=20)

gene_expression_df["log_Value"] = np.log(gene_expression_df["Value"].values)
sns.histplot(ax       = ax[1],
            data      = gene_expression_df,
            x         = "log_Value",
            hue       = "diagnosis",
            edgecolor = "w",
            bins    = 20,
            palette = "Set2")
ax[1].set_xlabel(None)
ax[1].set_ylabel(None)

for axi in ax.flatten():
    axi.spines["top"].set_visible(False)
    axi.spines["right"].set_visible(False)

fig.supxlabel("RNA expression", weight="bold")
fig.supylabel("count", weight="bold")


In [ ]:
np.unique(gene_expression2_df["diagnosis"], return_counts=True)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


gene_expression2_df = gene_expression_df[gene_expression_df.Quality>0]

fig, ax = plt.subplots(2, 1, figsize=(10.2, 9.2))

ax[0].hist(gene_expression2_df["Value"].values, edgecolor="w", facecolor="salmon",
                                                                alpha=0.5, bins=20)

gene_expression2_df["log_Value"] = np.log(gene_expression2_df["Value"].values)
sns.histplot(ax       = ax[1],
            data      = gene_expression2_df,
            x         = "Value",
            hue       = "diagnosis",
            edgecolor = "w",
            bins      = 20,
            stat      = "probability",
            palette   = "Set2")

ax[1].set_xlabel(None)
ax[1].set_ylabel(None)

for axi in ax.flatten():
    axi.spines["top"].set_visible(False)
    axi.spines["right"].set_visible(False)

fig.supxlabel("RNA expression", weight="bold")
fig.supylabel("count", weight="bold")


In [ ]:
def dig2num(diagnosis):
    if diagnosis == "DF":
        return 1
    elif diagnosis == "DHF":
        return 2
    elif diagnosis == "DSS":
        return 3
    else:
        return 0

logistic_reg_df = gene_expression2_df[["patient_ID", "mRNA", "Value", "diagnosis"]]
logistic_reg_df["value_diag"] = logistic_reg_df["diagnosis"].apply(dig2num)

labels_df                  = logistic_reg_df[["patient_ID", "value_diag"]].drop_duplicates().reset_index(drop=True)
labels_df["disease_label"] = labels_df["value_diag"].apply(lambda x: 1 if x>0 else 0)

expression_mat_df  = pd.pivot_table(logistic_reg_df, index="patient_ID", columns="mRNA", values="Value")


In [ ]:
# Importing library
from scipy.stats import f_oneway

healthy_df    = expression_mat_df.iloc[list(labels_df.disease_label==0), :]
diease_df     = expression_mat_df.iloc[list(labels_df.disease_label==1), :]

gene1_healthy = healthy_df.iloc[:, 0].values
gene1_disease = diease_df.iloc[:, 0].values


gene_anova_df = pd.DataFrame(columns=["gene", "pval", "gene_health_mean", "gene_disease_mean"])
for igene, gene in enumerate(expression_mat_df.columns):

    gene1_healthy = healthy_df.iloc[:, igene].dropna().values
    gene1_disease = diease_df.iloc[:, igene].dropna().values


    anov              = f_oneway(gene1_healthy, gene1_disease)
    pval_gene         = anov.pvalue

    gene_health_mean  = gene1_healthy.mean()
    gene_disease_mean = gene1_disease.mean()


    df = pd.DataFrame([{"gene" :        gene,
                                        "pval":              pval_gene,
                                        "gene_health_mean":  gene_health_mean,
                                        "gene_disease_mean": gene_disease_mean}])
    gene_anova_df = pd.concat([df, gene_anova_df], ignore_index=True)

gene_anova_df["log2_expression"] = np.log2(gene_anova_df["gene_disease_mean"]/gene_anova_df["gene_health_mean"])


fig, ax = plt.subplots(1, 1, figsize=(5.2, 5))

ax.scatter(gene_anova_df["log2_expression"], -np.log10(gene_anova_df["pval"]), fc="w", ec="r")
ax.axhline(-np.log10(0.05), color="k", ls="--")
ax.axvline(x=0, color="k", ls="--")

for gene in  gene_anova_df[gene_anova_df["pval"]<0.05].gene.values:
    row = gene_anova_df[gene_anova_df["gene"]==gene]

    ax.scatter(row.log2_expression,   -np.log10(row.pval), fc="r", ec="r")
    ax.text(row.log2_expression+1e-2, -np.log10(row.pval)+1e-2, gene)

#ax.text(-0.4, -np.log10(0.05), r"$\uparrow$ significant")

ax.set_xlabel(r"$\mathbf{{\log_2}}$ change in expression", weight="bold")
ax.set_ylabel(r"$\mathbf{{-\log_{10}}}$ p-value", weight="bold")

ax.set_title("Healthy vs Dengue disease", weight="bold")
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

ax.set_xlim(-np.max(np.abs(ax.get_xlim())), np.max(np.abs(ax.get_xlim())))

In [ ]:
gene_anova_df


In [ ]:
gene_anova_df

# PCA

In [ ]:
# X = feature values, all the columns except the last column
x_mat = expression_mat_df.iloc[:, :]
μ_x   = np.nanmean(x_mat, axis=0)
σ_x   = np.nanstd(x_mat, axis=0)

xn_mat = (x_mat - μ_x) / σ_x
xn_mat = np.c_[np.ones((xn_mat.shape[0], 1)), xn_mat] ## augment with column of ones
y_vec  = labels_df.iloc[:, -1].to_numpy()


In [ ]:
from sklearn.decomposition import PCA
pca_mRNA_denv = PCA(n_components=2, svd_solver='arpack')
pc_mRNA       = pca_mRNA_denv.fit_transform(np.nan_to_num(xn_mat[:, 1:]))


label2color = {0:"red", 1:"black"}
y_color     = [label2color[y] for y in y_vec]

pca_df          = pd.DataFrame(data = pc_mRNA, columns = ['pc1', 'pc2'])
pca_df["color"] = y_color

fig, ax = plt.subplots(1, 1, figsize=(5.2, 5))

labels = ["Dengue disease", "healthy"]
for ic, c in enumerate(["red", "black"]):

    pca_plot_df = pca_df[pca_df.color == c]
    ax.scatter(pca_plot_df.pc1, pca_plot_df.pc2, color=c, label=labels[ic])

ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

ax.legend(loc="upper left", bbox_to_anchor=(0, 1.15), ncol=2, frameon=False)

exp_var = pca_mRNA_denv.explained_variance_

ax.set_xlabel("principal component 1, {:0.2f}%".format(exp_var[0]), weight="bold")
ax.set_ylabel("principal component 2, {:0.2f}%".format(exp_var[1]), weight="bold")


In [ ]:
pca_mRNA_denv = PCA(n_components=20, svd_solver='arpack')
pc_mRNA       = pca_mRNA_denv.fit_transform(np.nan_to_num(xn_mat[:,1:]))

dfLoadings = pd.DataFrame(pca_mRNA_denv.components_,
                          columns = expression_mat_df.columns,
                          index   = [f"pc{i}" for i in range(1, 20+1)])


In [ ]:
pca_mRNA_denv.explained_variance_

fig, ax = plt.subplots(1, 1, figsize=(7.2, 5))

ax.bar(range(1,     len(pca_mRNA_denv.explained_variance_ratio_)+1), pca_mRNA_denv.explained_variance_, color="teal")
ax.plot(range(1,    len(pca_mRNA_denv.explained_variance_ratio_)+1), np.cumsum(pca_mRNA_denv.explained_variance_), color="k", ls="-")
ax.scatter(range(1, len(pca_mRNA_denv.explained_variance_ratio_)+1), np.cumsum(pca_mRNA_denv.explained_variance_), color="k")

ax.set_ylabel("explained variance %", weight="bold")
ax.set_xlabel("principal component", weight="bold")

ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)


# Logistic regression

In [ ]:
from scipy.optimize import fmin_tnc

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def prob(theta, x):
    return sigmoid(np.dot(x, theta))

def objective(theta, x, y):
    # Computes the (negative of the) objective function, for all the training samples
    p = prob(theta, x)
    return -np.sum(y * np.log(p) + (1 - y) * np.log(1 - p))

def gradient(theta, x, y):
    # Computes the gradient of the cost function at the point theta values
    return np.dot(x.T, sigmoid(np.dot(x, theta)) - y)

def fit(x, y, theta):
    return fmin_tnc(func=objective, x0=theta, fprime=gradient, args=(x, y))[0]


In [ ]:
θ_0        = np.zeros((xn_mat.shape[1], 1))
theta_star = fit(np.nan_to_num(xn_mat), y_vec, θ_0)
theta_star

In [ ]:
def accuracy(x, actual_classes, θ):
    predicted_classes = (prob(θ, x) >= 0.5).astype(int).flatten()
    return 100 * np.mean(predicted_classes == actual_classes)

acc = accuracy(xn_mat, y_vec, theta_star)

print("\n Accuracy {}\n".format(acc))

In [ ]:

fig, ax = plt.subplots(1, 1, figsize=(7.2, 5))
ax.bar(expression_mat_df.columns.values, theta_star[1:], color="gray", edgecolor="w")

ax.axhline(0, color="k", ls="-", lw=0.5)

ax.set_ylabel(r"logistic regression coefficient, $\mathbf{{\theta_i}}$", weight="bold")
ax.set_xticklabels(expression_mat_df.columns.values, rotation=90)
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

## logistic regression just on the 5 genes significant ($p_{\text{value}}$ $\leq$ 0.05 ) in the ANOVA (volcano plot)


In [ ]:
genes_diff_exp = ["GAPDH", "TOR3A", "GYG1", "RABL2A", "CACNA2D3"]

x_mat = expression_mat_df[genes_diff_exp].iloc[:, :]
μ_x   = np.nanmean(x_mat, axis=0)
σ_x   = np.nanstd(x_mat, axis=0)

xn_mat = (x_mat - μ_x) / σ_x
xn_mat = np.c_[np.ones((xn_mat.shape[0], 1)), xn_mat] ## augment with column of ones
y_vec  = labels_df.iloc[:, -1].to_numpy()

# regression
θ_0        = np.zeros((xn_mat.shape[1], 1))
θ_star = fit(np.nan_to_num(xn_mat), y_vec, θ_0)
θ_star

acc   = accuracy(xn_mat, y_vec, θ_star)
print("\n Accuracy {}\n".format(acc))


## logistic regression on the first 20 principal components

In [ ]:
θ_02  = np.zeros((pc_mRNA.shape[1], 1))
θ_hat = fit(pc_mRNA, y_vec, θ_02)


In [ ]:
acc   = accuracy(pc_mRNA, y_vec, θ_hat)
print("\n Accuracy {}\n".format(acc))


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7.2, 5))
ax.bar([f"PC{i}" for i in range(1, 20)], θ_hat[1:], color="gray", edgecolor="w")

ax.set_ylabel(r"logistic regression coefficient, $\mathbf{{\theta_i}}$", weight="bold")

ax.set_xticklabels([f"PC{i}" for i in range(1, 20)], rotation=90)
ax.axhline(0, color="k", ls="-", lw=0.5)
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)


## cross-validation (and bootstrapping)

textbooks typically introduce both cross-validation and bootstrapping in different sections. They technically are different but the goal of both is the same: estimate expected prediction error.


**cross-validation**

in CV one ideally has a huge dataset that can split in $K$ partitions each one with the same number of training samples. Given those partitions, the training procedure involves training a model in the data that belongs to $K-1$ partitions and using the one left out for testing. That way you'll end with $K$ models trained and the expected prediction accuracy is just the average accuracy across the partitions.

**bootstrapping**

Bootstrapping corresponds to a sampling procedure of a fixed dataset as described below. I like to think of this method as a Monte Carlo method where the distribution to sample is the empirical distribution given by the dataset and thus random sampling is equivalent to drawing from that distribution. As a consequence, the estimate of the prediction accuracy also has embedded information of the variance of that accuracy, a Monte Carlo estimate specifically. Thus, cross-validation allows an estimate of the variance of the prediction accuracy.


Additionally, typically it is hard to have huge sample sizes of observed phenomena, like the Dengue disease case where clinical cases due to severe disease are a small ($\sim$ 10\%) sample of the true infections. and thus the bootstrapping language is preferred.


the idea here is to created $K$ models trained on $N-T$ train samples where $N$ is the size of the data set and $T$ the size of the test set.


the $N-T$ train samples are drawn with resample from the dataset, so each training set represents a random population sample and so does the test set.


in reality of course is not random as $P(\text{blood sample}|\text{disease})$ is way bigger than $P(\text{blood sample}|\text{healthy})$ in real life, but we don't know those numbers so far so bootstrapping (cross-validation) is the best we can do to estimate the test error/accuracy.


This estimate is in theory a statistical, unbiased $\left(\mathbb{E}[\hat{\text{error}}] = \text{error}\right)$, estimate of the test error in IID population, which is generally not the case but still is the best we can do.



In [ ]:
def cross_val_indexes(n, n_train_frac=80, K=30):

    n_train = int(n * n_train_frac / 100)
    n_test  = n - n_train

    train_indexes = np.full((K, n_train), np.nan)
    test_indexes  = np.full((K, n_test), np.nan)

    indexes = np.arange(n)
    for ki in range(K):

        train_indexes[ki, :] = np.random.choice(indexes, n_train, replace=True)
        test_index           = list(set(indexes) - set(train_indexes[ki, :]))
        test_indexes[ki, :]  = np.random.choice(test_index, n_test, replace=True)

    return np.asarray(train_indexes, dtype=int),  np.asarray(test_indexes, dtype=int)



In [ ]:
x_mat = expression_mat_df.iloc[:, :]
μ_x   = np.nanmean(x_mat, axis=0)
σ_x   = np.nanstd(x_mat, axis=0)

xn_mat = (x_mat - μ_x) / σ_x
xn_mat = np.c_[np.ones((xn_mat.shape[0], 1)), xn_mat] ## augment with column of ones
y_vec  = labels_df.iloc[:, -1].to_numpy()

K   = 300
train_indexes, test_indexes = cross_val_indexes(n=len(expression_mat_df), n_train_frac=80, K=K)

θ0 = np.zeros((xn_mat.shape[1], 1))

θ_cv       = np.full((K, xn_mat.shape[1]), np.nan)
accu_train = np.full(K, np.nan)
accu_test  = np.full(K, np.nan)

for ki in range(K):

    train_set = train_indexes[ki, :]
    test_set  = test_indexes[ki, :]

    xn_train  = np.nan_to_num(xn_mat[train_set, :])
    y_train   = y_vec[train_set]

    xn_test  = np.nan_to_num(xn_mat[test_set, :])
    y_test   = y_vec[test_set]

    θ           = fit(xn_train, y_train, θ0)
    θ_cv[ki, :] = θ

    acc_train = accuracy(xn_train, y_train, θ)
    acc_test  = accuracy(xn_test, y_test, θ)

    accu_train[ki] = acc_train
    accu_test[ki]  = acc_test


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7.2, 5))

bplot = ax.boxplot(θ_cv[:, :],
                patch_artist = True,  # fill with color
                labels       = ["intercept"]+list(expression_mat_df.columns.values))  # will be used to label x-ticks

ax.set_ylabel(r"logistic regression coefficient, $\mathbf{{\theta_i}}$", weight="bold")
ax.set_xticklabels(["intercept"]+list(expression_mat_df.columns.values), rotation=90)
ax.axhline(0, color="k", ls="-", lw=0.5)
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

# fill with colors.
for patch in bplot['boxes']:
    patch.set_facecolor("w")
    patch.set_edgecolor("k")


In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(9.2, 5))

bplot1 = ax[0].boxplot(accu_train,
                    patch_artist = True,          # fill with color
                    labels       = ["train"])     # will be used to label x-ticks
bplot2 = ax[1].boxplot(accu_test,
                        patch_artist = True,      # fill with color
                        labels       = ["test"])  # will be used to label x-ticks

ax[2].scatter(accu_train, accu_test, fc="w", ec="k")

for axi in ax.flatten():
    axi.spines["right"].set_visible(False)
    axi.spines["top"].set_visible(False)

# fill with colors
for patch in bplot1['boxes']:
    patch.set_facecolor("w")
    patch.set_edgecolor("k")

for patch in bplot2['boxes']:
    patch.set_facecolor("w")
    patch.set_edgecolor("k")

ax[2].set_xlabel("train accuracy %", weight="bold")
ax[2].set_ylabel("test accuracy %", weight="bold")

fig.supylabel("accuracy %", weight="bold")